In [1]:
import numpy as np
import pandas as pd
import scipy.stats as  st

pd.options.display.max_rows = 100

Как всегда прежде всего взглянем на данные

In [2]:
df_FB = pd.read_csv('dataset_Facebook.csv', sep=';')
df_FB.head()

,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
0,139441,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100
1,139441,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164
2,139441,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80
3,139441,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1777
4,139441,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,393


Видим, что не все столбцы имеют типы, подходящие для вычисления средних, медиан и тд. Отберем нужные нам столбцы в отдельный лист.

In [3]:
nes_columns = [col for col, typ in df_FB.dtypes.to_dict().items() if typ != 'O']

### Вычисление метрик для каждого типа контента для каждого столбца

Можно сделать аггрегации для каждого столбца, предварительно сделав group by по type:

In [4]:
df_FB.groupby('Type')[nes_columns].agg({'mean':   'mean', 
                                        'max':    'max', 
                                        'min':    'min', 
                                        'median': 'median', 
                                        'mode':   lambda x: st.mode(x).mode[0]}).T

C:\Users\Tager\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\groupby.py:4291: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


Type                                                                Link  \
mode   like                                                    12.000000   
       Lifetime Post Total Impressions                       3094.000000   
       Lifetime Post Impressions by people who have li...    2307.000000   
       Category                                                 1.000000   
       Post Weekday                                             6.000000   
       Lifetime Post Consumers                                322.000000   
       comment                                                  0.000000   
       Total Interactions                                      14.000000   
       share                                                    2.000000   
       Lifetime Engaged Users                                  66.000000   
       Lifetime Post reach by people who like your Page      1180.000000   
       Page total likes                                    138353.000000   
       Lifetime Post Consumptions                              26.000000   
       Lifetime People who have liked your Page and en...      19.000000   
       Post Hour                                                3.000000   
       Lifetime Post Total Reach                             1536.000000   
       Post Month                                              12.000000   
       Paid                                                     0.000000   
min    like                                                     5.000000   
       Lifetime Post Total Impressions                       3094.000000   
       Lifetime Post Impressions by people who have li...    2307.000000   
       Category                                                 1.000000   
       Post Weekday                                             1.000000   
       Lifetime Post Consumers                                 23.000000   
       comment                                                  0.000000   
       Total Interactions                                       6.000000   
       share                                                    0.000000   
       Lifetime Engaged Users                                  24.000000   
       Lifetime Post reach by people who like your Page      1180.000000   
       Page total likes                                     85979.000000   
       Lifetime Post Consumptions                              26.000000   
       Lifetime People who have liked your Page and en...      19.000000   
       Post Hour                                                2.000000   
       Lifetime Post Total Reach                             1536.000000   
       Post Month                                               1.000000   
       Paid                                                     0.000000   
median like                                                    37.000000   
       Lifetime Post Total Impressions                       9773.000000   
       Lifetime Post Impressions by people who have li...    9034.500000   
       Category                                                 1.000000   
       Post Weekday                                             4.500000   
       Lifetime Post Consumers                                205.000000   
       comment                                                  1.500000   
       Total Interactions                                      52.500000   
       share                                                   10.500000   
       Lifetime Engaged Users                                 244.000000   
       Lifetime Post reach by people who like your Page      5039.000000   
       Page total likes                                    115396.000000   
       Lifetime Post Consumptions                             290.000000   
       Lifetime People who have liked your Page and en...     161.500000   
       Post Hour                                                4.000000   
       Lifetime Post Total Reach                

### Вычисление метрик для каждого типа контента для всего массива

In [5]:
df_FB.groupby('Type').apply(lambda x: pd.Series({'Mean':   np.nanmean  (x[nes_columns].values), 
                                                 'Median': np.nanmedian(x[nes_columns].values),
                                                 'Max':    np.nanmax   (x[nes_columns].values),
                                                 'Min':    np.nanmin   (x[nes_columns].values),
                                                 'Mode':   st.mode     (x[nes_columns].values, axis=None, nan_policy='omit').mode[0]}))

,Max,Mean,Median,Min,Mode
Type,,,,,
Link,229733.0,10152.356061,76.5,0.0,1.0
Photo,1110282.0,10604.856173,234.0,0.0,1.0
Status,139441.0,11530.497531,270.0,0.0,2.0
Video,277100.0,19825.865079,448.0,0.0,1.0


### Для всего массива

Поскольку во фрейме есть пропуски, то не совсем корректно будет считать метрики стандартными функциями pandas. Переведем в numpy массив и воспользуемся функциями numpy для таких случаев:

In [6]:
df_FB.groupby(lambda _: True).apply(lambda x: pd.Series({'Mean':   np.nanmean  (x[nes_columns].values), 
                                                         'Median': np.nanmedian(x[nes_columns].values),
                                                         'Max':    np.nanmax   (x[nes_columns].values),
                                                         'Min':    np.nanmin   (x[nes_columns].values),
                                                         'Mode':   st.mode     (x[nes_columns].values, axis=None, nan_policy='omit').mode[0]}))

,Max,Mean,Median,Min,Mode
True,1110282.0,10797.476429,224.5,0.0,1.0


Наиболее часто встречающимся элементом в массиве будет 1, поскольку мода по всему массиву = 1. 